In [1]:
from IPython.display import display_html
display_html("""<button onclick="$('.input, .prompt, .output_stderr, .output_error').toggle();">Toggle Code</button>""", raw=True)

Toggle Code

# OpenStreetMap
***Data Wrangling with mongoDB by NK Zhehua Zou***
  
Map Area: San Jose, CA, United States  
https://mapzen.com/data/metro-extracts/metro/san-jose_california/  
  
***Table of Contents***
1. Data Audit
2. Problems Encountered in the Map  
Abbreviated Street Names  
Postal Codes  
3. Data Overview  
4. Additional Ideas  
Contributor statistics and gamification suggestion  
Additional data exploration using MongoDB  
5. Conclusion

# 1. Data Audit

In [2]:
# Load packages and libraries
import xml.etree.cElementTree as ET
import re
import collections
from collections import defaultdict
import pymongo
from pymongo import MongoClient
import os
import json
import pprint
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Load data
# This data just a sample for code testing, I didn't change analysis from original data
# Please read html file if you want to reviewed entire analysis.
filename = 'sample.osm'

### Tags
Parse through the San Jose dataset with ElementTree and count the number of unique element types to get an overall understanding of the data by using count_tags function.

In [4]:
# This function will takes 5~10 seconds. Be patient on on next step.
# Parse through the data with ElementTree.
def count_tags(filename):
    tags={}
    for event, elem in ET.iterparse(filename):
        if elem.tag in tags:
            tags[elem.tag]+=1
        else:
            tags[elem.tag]=1
    return tags

In [5]:
count_tags(filename)

{'bounds': 1,
 'member': 115,
 'nd': 21564,
 'node': 13332,
 'osm': 1,
 'relation': 21,
 'tag': 7128,
 'way': 3721}

### Keys Type
*** For the follinwg function: key_type & process_map. We check the "k" value for each. ***  
"lower", for tags that contain only lowercase letters and are valid.  
"lower_colon", for otherwise valid tags with a colon in their names.  
"problemchars", for tags with problematic characters.

In [6]:
# Count of each of three tag categories in a dictionary with re.
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
# This regex represents invalid MongoDB characters for keys.
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    if element.tag == 'tag':
        if re.match(lower,element.get('k'))!=None:
            keys['lower']+=1
        elif re.match(lower_colon,element.get('k'))!=None:
            keys['lower_colon']+=1
        elif re.match(problemchars,element.get('k'))!=None:
            keys['problemchars']+=1
        else:
            keys['other']+=1
    return keys

def process_map(filename):
    keys = {'lower': 0, 'lower_colon': 0, 'problemchars': 0, 'other': 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)
    return keys

In [7]:
process_map(filename)

{'lower': 6751, 'lower_colon': 363, 'other': 14, 'problemchars': 0}

### Users

In [8]:
def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        for e in element:
            if 'uid' in e.attrib:
                users.add(e.attrib['uid'])
    return users

print str(len(process_map(filename))) + ' peoples invovlved in the map editing.'

94 peoples invovlved in the map editing.


# 2. Problems Encountered in the Map
After initially downloading a small sample size of the San Jose area and running it, I noticed three main problems with the data, which I will discuss in the following order:  
1) Abbreviated street names ('Branham Ln')  
2) Inconsistent postal codes ('CA950543', '95014-1899')

### Abbreviated Street Names
Once the data was imported to MongoDB, some basic querying revealed street name abbreviations. I updated all substrings in problematic address strings, such that 'Branham Ln' becomes 'Branham Lane'.

1) The main problem we encountered in this dataset come from the street name abbreviation inconsistency. We build the regex matching the last element in the string, where usually the street type is based. Then we come up with a list of mapping that need not to be cleaned.  
2) audit_street_type function search the input string for the regex. If there is a match and it is not within the 'expected' list, add the match as a key and add the string to the set.  
3) is_street_name function looks at the attribute k if k='addre:street'.  
4) audit functio will return the list that match previous two functions.  
5) After that, we would do a pretty print the output of the audit. With the list of all the abbreviated street types we can understand and fill-up our 'mapping' dictionary as a preparatio to convert these street name into proper form. (list of 1)  
6) update_name is the last step of the process, which take the old name and update them with a better name. (list of 2)

In [9]:
from clean_street import audit
from clean_street import update_street
from clean_street import mapping
from clean_street import street_type_re

for street_type, ways in audit(filename).iteritems():
    for name in ways:
        better_name = update_street(name, mapping, street_type_re)
        print name, '=>', better_name

San Jose => San Jose
Arica => Arica
Las Diamelas => Las Diamelas
Maria Izaga => Maria Izaga
Elias Aguirre => Elias Aguirre
Avenida Luis Gonzales => Avenida Luis Gonzales
Yahuar Huaca => Yahuar Huaca
Simon Bolivar => Simon Bolivar
Salaverry => Salaverry
Av. Jose Balta => Av. Jose Balta
Leoncio Prado => Leoncio Prado
Conquista => Conquista
Av. Pedro Ruiz => Av. Pedro Ruiz
la Libertad => la Libertad
Francisco Cabrera => Francisco Cabrera
Husares de Junin => Husares de Junin
Alfonso Ugarte => Alfonso Ugarte
Paul Harris => Paul Harris
Loreto => Loreto
Manuel Seoane => Manuel Seoane
Congreso => Congreso
Juan Tomis Stack => Juan Tomis Stack
Los Amautas => Los Amautas
El Eden => El Eden
Panamericana norte => Panamericana norte
Sáenz Peña => Sáenz Peña
Elvira Garcia Y Garcia => Elvira Garcia Y Garcia
Los Andes => Los Andes
Avenida Oriente => Avenida Oriente
Pasaje manuel seoane => Pasaje manuel seoane
eduardo mesa => eduardo mesa
Av. Angamos => Av. Angamos
Zona Industrial => Zona Industrial
Las

### Postal Codes
Postal code strings posed a different sort of problem, forcing a decision to strip all leading and trailing characters before and after the main 5-digit zip code. This effectually dropped all leading state characters (as in 'CA950543') and 4-digit zip code extensions following a hyphen ('95014-1899'). This 5-digit constriction benefits MongoDB aggregation calls on postal codes.  
1) Although most of the zip code is correct, there're still a lot of zip code with incorrect 5 digit formats. We will process it like update street name. (list of 1)  
2 )The output of the clean zip code is summarised below. There are the format of 5 digits. (list of 2)

In [10]:
from clean_postal import audit_zip

def update_postal(zipcode):
    testNum = re.findall('[a-zA-Z]*', zipcode)
    if testNum:
        testNum = testNum[0]
    testNum.strip()
    if testNum == 'CA':
        convertedZipcode = (re.findall(r'\d+', zipcode))
        if convertedZipcode:
            if convertedZipcode.__len__() == 2:
                return (re.findall(r'\d+', zipcode))[0] + "-" +(re.findall(r'\d+', zipcode))[1]
            else:
                return (re.findall(r'\d+', zipcode))[0]
    elif re.match(r'^95\d+', zipcode):
        return re.findall(r'\d{5}', zipcode)[0]

for street_type, ways in audit_zip(filename).iteritems():
    for name in ways:
        better_name = update_postal(name)
        print name, '=>', better_name

140101 => None
14820 => None
074 => None


# 3. Data Overview
This section contains basic statistics about the dataset and the MongoDB queries used to gather them.  
  
### Preparing for MongoDB by converting XML to JSON
In order to transform the data from XML to JSON, we need to follow these rules:  
1) Process only 2 types of top level tags: "node" and "way"  
2) All attributes of "node" and "way" should be turned into regular key/value pairs, except:   attributes in the CREATED array should be added under a key "created", attributes for latitude and longitude should be added to a "pos" array, for use in geospacial indexing. Make sure the values inside "pos" array are floats and not strings.  
3) If second level tag "k" value contains problematic characters, it should be ignored  
4) If second level tag "k" value starts with "addr:", it should be added to a dictionary "address"  
5) If second level tag "k" value does not start with "addr:", but contains ":", you can process it same as any other tag.  
6) If there is a second ":" that separates the type/direction of a street, the tag should be ignored  
After all the cleaning and data transformation are done, we would use last function process_map and convert the file from XML into JSON format

In [11]:
client = MongoClient()
db=client.project

In [12]:
db.doc.drop()

In [13]:
from prepare import process_map
db.doc.insert_many(process_map(filename));

### File sizes

In [14]:
print 'The original OSM file is ' + str(os.path.getsize(filename)/1.0e6) + ' MB'

The original OSM file is 3.329866 MB


In [15]:
print 'The JSON file is ' + str(os.path.getsize(filename + '.json')/1.0e6) + ' MB'

The JSON file is 3.795993 MB


In [16]:
# Number of documents, we defined it for next section.
number_document = db.doc.find().count()
print 'The number of documents is ' + str(number_document)

The number of documents is 17053


In [17]:
# Number of nodes
print 'The number of node is ' + str(db.doc.find({'type':'node'}).count())

The number of node is 13332


In [18]:
# Number of ways
print 'The number of way is ' + str(db.doc.find({'type':'way'}).count())

The number of way is 3721


In [19]:
# Number of unique users, we defined it for next section.
number_unique_users = len(db.doc.distinct('created.user'))
print 'The number of unique users is ' + str(number_unique_users)

The number of unique users is 91


In [20]:
# Top 1 contributing user
cursor = db.doc.aggregate([{'$group':{'_id':'$created.user', 'count':{'$sum':1}}}, {'$sort':{'count':-1}}, {'$limit':1}])
for res in cursor:
    user1=res['_id']
    user1_count=res['count']
print 'The first contributor is ' + user1 + ' with '+ str(user1_count) + ' contributions.'

The first contributor is TELCOM IP with 4921 contributions.


In [21]:
# Number of users appearing only once (having 1 post), we defined it for next section.
user_once=db.doc.aggregate([{'$group':{'_id':'$created.user', 'count':{'$sum':1}}}, 
                       {'$sort':{'count':1}},
                       {'$match':{'count':1}},
                       {'$group':{'_id':'null','total':{'$sum':'$count'}}}
                        ])
for res in user_once:
    number_user_once=res['total']

print 'There is ' + str(number_user_once) + ' users appearing only once.'

There is 22 users appearing only once.


# 4. Additional Ideas
### Contributor statistics and gamification suggestion
The contributions of users seems incredibly skewed, possibly due to automated versus manual map editing (the word “bot” appears in some usernames). Here are some user percentage statistics:  
1) Top1 Contributor is TELCOM IP, contribution percentage is 28%.  
2) Combined top 2 users are [u'TELCOM IP', u'negro'], contribution percentage is 42%.  
3) Combined top 10 users are [u'TELCOM IP', u'negro', u'Diego Sanguinetti', u'pizza4days', u'ovruni', u'WorstFixer',   u'greecemapper', u'Paper_', u'MintCondition', u'dbusse'], contribution percentage is 88%.  
4) 24% of users contribute with one post.    
5) Thinking about these user percentages from this graph below, I’m reminded of “gamification” as a motivating force for contribution. In the context of the OpenStreetMap, if user data were more prominently displayed, perhaps others would take an initiative in submitting more edits to the map. And, if everyone sees that only 10 of power users are creating more than 88% a of given map, that might spur the creation of more efficient bots, especially if certain gamification elements were present, such as rewards, badges, or a leaderboard.

In [22]:
from top_contribution import topn_contrib

top1,top1_percent_contrib=topn_contrib(1,user=True)
print 'Top1 Contributor is ' + top1[0] + ', contribution percentage is ' + str(top1_percent_contrib) + '%.'
top2,top2_percent_contrib=topn_contrib(2,user=True)
print 'Combined top 2 users are ' + str(top2) + ', contribution percentage is ' + str(top2_percent_contrib) + '%.'
top10,top10_percent_contrib=topn_contrib(10, user=True)
print 'Combined top 10 users are ' + str(top10) + ', contribution percentage is ' + str(top10_percent_contrib) + '%.'
percent_user_1post=(number_user_once*100)/number_unique_users
print str(percent_user_1post) + '% of users contribute with one post.'

Top1 Contributor is TELCOM IP, contribution percentage is 28%.
Combined top 2 users are [u'TELCOM IP', u'negro'], contribution percentage is 42%.
Combined top 10 users are [u'TELCOM IP', u'negro', u'Diego Sanguinetti', u'pizza4days', u'ovruni', u'WorstFixer', u'greecemapper', u'Paper_', u'MintCondition', u'dbusse'], contribution percentage is 88%.
24% of users contribute with one post.


### Additional data exploration using MongoDB queries
1) Top 10 appearing amenities

In [23]:
amenity = db.doc.aggregate([{'$match':{'amenity':{'$exists':1}}},
                               {'$group':{'_id':'$amenity', 'count':{'$sum':1}}},
                               {'$sort':{'count':-1}},
                               {'$limit':10}])

print list(amenity)

[{u'count': 22, u'_id': u'bus_station'}, {u'count': 12, u'_id': u'fuel'}, {u'count': 12, u'_id': u'school'}, {u'count': 7, u'_id': u'restaurant'}, {u'count': 5, u'_id': u'hospital'}, {u'count': 5, u'_id': u'place_of_worship'}, {u'count': 4, u'_id': u'marketplace'}, {u'count': 4, u'_id': u'veterinary'}, {u'count': 4, u'_id': u'townhall'}, {u'count': 4, u'_id': u'university'}]


2) Biggest religion

In [24]:
biggest_religion = db.doc.aggregate([{'$match':{'amenity':{'$exists':1}, 'amenity':'place_of_worship'}},
                    {'$group':{'_id':'$religion', 'count':{'$sum':1}}},
                    {'$sort':{'count':-1}}, {'$limit':1}])

print list(biggest_religion)

[{u'count': 3, u'_id': u'christian'}]


3) Most popular cuisines

In [25]:
popular_cuisines = db.doc.aggregate([{'$match':{'amenity':{'$exists':1}, 'amenity':'restaurant'}}, 
                    {'$group':{'_id':'$cuisine', 'count':{'$sum':1}}},
                    {'$sort':{'count':-1}}, {'$limit':1}])

print list(popular_cuisines)

[{u'count': 4, u'_id': None}]


# 5. Conclusion
1) The map about the city of San Jose is relatively clean so I could retrieve some interesting content. But still the data is not entirely clean.  
2) The data contains some mistakes or different references for the same feature. So I had to clean the data programmatically for the street and the postal codes.  
3) When we audit the data, it was very clear that although there are minor error caused by human input, the dataset is fairly well-cleaned. Considering there're hundreds of contributors for this map, there is a great numbers of human errors in this project. I'd recommend a srtuctured input form so everyone can input the same data format to reduce this error.  
4) We can incentivize users by gamify the contribution process, then we can create a recommendation engine to leverage these data (eg. restaurant recommendation, building, etc).  
5) OpenStreetMaps is an open source project, there're still a lot of areas left unexplored as people tend to focus on a certain key areas and left other part outdated. Since each node has a coordinate (lattitude & longtitude), we can resolve this issue by cross-referencing/cross-validating missing data from other database like Google API.

##### References
1) https://github.com/GuillaumeSalvan/P3-Wrangle-OpenStreetMap-Data  
2) https://github.com/lyvinhhung/Udacity-Data-Analyst-Nanodegree/tree/master/p3%20-%20Wrangle%20OpenStreetMap%20Data